## The forward and backward passes

In [39]:
#export
from pathlib import Path
from IPython.core.debugger import set_trace
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor
import operator
from pathlib import Path
import numpy as np
import torch.nn.functional as F

from torch.nn import init

def near(a,b): return torch.allclose(a, b, rtol=1e-3, atol=1e-5)
def test_near(a,b): test(a,b,near)
def test(a,b,cmp,cname=None):
    if cname is None: cname=cmp.__name__
    assert cmp(a,b),f"{cname}:\n{a}\n{b}"

def test_eq(a,b): test(a,b,operator.eq,'==')

def get_data():
    path = Path('./mnist.pkl.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

$    \lvert \text{input} - \text{other} \rvert \leq \texttt{atol} + \texttt{rtol} \times \lvert \text{other} \rvert
$

In [2]:
x_train,y_train,x_valid,y_valid = get_data()

In [3]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [4]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [5]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.0001), tensor(1.))

In [6]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [7]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [8]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

In [9]:
aa = np.array([1,2,3,4,5,])

## Foundations version

### Basic architecture

In [10]:
# num hidden
nh = 50

In [11]:
np.random.randn(m, nh)

array([[ 0.60643784,  1.38366484, -2.27439277, ..., -0.33923247,
         0.97545337, -0.9949854 ],
       [ 0.86839103, -0.97640685,  0.74690988, ...,  0.03089841,
         0.52312458,  0.43254768],
       [-1.4650711 ,  0.75735995,  1.18590432, ...,  1.29115752,
         1.03018806, -0.03370648],
       ...,
       [ 0.17131332, -0.66671828,  0.00268098, ...,  0.22426993,
        -1.48836245,  1.10912227],
       [-0.06615231,  0.89421131,  1.71322553, ..., -1.5402347 ,
        -1.19326853, -0.942686  ],
       [ 0.22685865,  0.28243727, -1.00657977, ..., -0.35613913,
        -1.77672123, -0.53874639]])

In [12]:
np.zeros(nh)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [13]:
w1 = torch.randn(m,nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)
b2 = torch.zeros(1)

In [14]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [15]:
def lin(x, w, b): return x@w + b

In [16]:
t = lin(x_valid, w1, b1)

In [17]:
def relu(x): return x.clamp_min(0.)

In [18]:
t = relu(lin(x_valid, w1, b1))

In [19]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [20]:
%timeit -n 10 _=model(x_valid)

3.93 ms ± 318 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function: MSE

In [22]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [23]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [24]:
y_train,y_valid = y_train.float(),y_valid.float()

In [25]:
preds = model(x_train)

In [26]:
preds.shape

torch.Size([50000, 1])

In [27]:
mse(preds, y_train)

tensor(8288.0195)

### Gradients and backward pass

In [28]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [29]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [58]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    print(inp.shape)
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    print(w.g.shape)
    b.g = out.g.sum(0)

In [59]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [33]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

In [34]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [35]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [36]:
loss = forward(xt2, y_train)

In [37]:
loss.backward()

In [40]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor model

### Layers as classes

In [50]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [51]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [52]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [53]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [54]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [55]:
%time loss = model(x_train, y_train)

CPU times: user 155 ms, sys: 2.06 ms, total: 157 ms
Wall time: 27.2 ms


In [56]:
%time model.backward()

CPU times: user 6.56 s, sys: 1.86 s, total: 8.41 s
Wall time: 7.04 s


In [57]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)